# Introduction

This notebook presents an exploratory analysis of Wintermute’s on-chain trading activity within decentralised finance. The dataset consists of raw transfer records which are enriched using the `DeFiProtocolClassifier`, a tool that identifies whether each counterparty corresponds to a DeFi protocol, a centralised exchange, or an unknown entity. This classification enables a clearer understanding of the types of platforms Wintermute interacts with and the broader patterns underlying its transactional behaviour.

The analysis proceeds through the following steps:

1. Load the Wintermute transfer dataset.
2. Apply the protocol classification model to each counterparty address.
3. Construct a filtered dataset containing only Wintermute’s interactions with DeFi protocols.
4. Examine the distribution and characteristics of these DeFi interactions.

This framework provides a basis for assessing Wintermute’s operational footprint in DeFi, identifying liquidity routes, and evaluating behavioural trends within its on-chain activity.



# Import Libraries and Dataframe

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from ipywidgets import interact, Dropdown



sys.path.append(str(Path().resolve().parent / "src"))
from defiClassifier import DeFiProtocolClassifier

data_file = Path("..") / "data" / "wintermute_transfers_search_default_2025-04-08.csv"
df = pd.read_csv(data_file)

df.head()

,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number
0,2025-04-08 12:49:57+00:00,0xB1026b8e7276e7AC75410F1fcbbe21796e8f7526,Camelot,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0
1,2025-04-08 12:49:54+00:00,0xcDa53B1F66614552F834cEeF361A8D12a0B8DaD8,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59,Aerodrome Finance,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0
3,2025-04-08 12:49:51+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,0x72AB388E2E2F6FaceF59E3C3FA2C4E29011c2D38,PancakeSwap,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0
4,2025-04-08 12:49:50+00:00,0x641C00A822e8b671738d32a431a4Fb6074E5c79d,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0


# Generating a List of All Counterparties Interacting with Wintermute
In this section, we extract and consolidate all unique counterparty addresses appearing in the Wintermute transfer dataset to establish the universe of entities Wintermute has interacted with on-chain.


In [3]:
wm_from = df[df["from_entity"]=="Wintermute"]
wm_to = df[df["to_entity"]=="Wintermute"]

cp_from = wm_from["to_entity"].fillna(wm_from["to_address"])
cp_to = wm_to["from_entity"].fillna(wm_to["from_address"])

counterparties = pd.concat([cp_from, cp_to]).dropna().unique().tolist()

counterparties[:10], len(counterparties)

(['PancakeSwap',
  'Uniswap',
  '0xCB43d843F6cAdF4F4844F3F57032468aAdd9B95c',
  'CoW Protocol',
  '0x3B55732F6D3997A7D44A041b8496e1A60712A35F',
  'Fluid (Instadapp)',
  'Orca',
  'GsWhFTWKNAxo8jmLhW7PZNDnMBa9ipyr8UMveF4QBLte',
  'Aerodrome Finance',
  'Raydium'],
 4419)

### Classifying Counterparties with `DeFiProtocolClassifier`

With the list of unique counterparty identifiers prepared, the next step is to determine which of these entities correspond to DeFi protocols. The `DeFiProtocolClassifier` provides a scoring-based matching system that compares each counterparty against the DeFiLlama protocol registry.

The classifier is first initialised, after which the `diagnose` method is applied directly to the list of counterparties:



In [4]:
clf = DeFiProtocolClassifier()
results = clf.diagnose(counterparties)


### Constructing a DeFi-Only Interaction Dataset

The classification results are now integrated back into the transaction dataset. Each row is first assigned a `counterparty` field, identifying the entity interacting with Wintermute in either the sender or receiver position. The list of classification outputs is then converted into a DataFrame and merged onto the main dataset using this counterparty identifier.

After the merge, the dataset is filtered to retain only those rows where the matched entity is classified as a DeFi protocol. The resulting `df_defi` table contains all transactions in which Wintermute directly interacted with a DeFi platform, forming the basis for subsequent analysis.


In [5]:
# Convert results list to DataFrame
results_df = pd.DataFrame(results)

# Build counterparty column
df["counterparty"] = df.apply(
    lambda r: r["to_entity"] if r["from_entity"]=="Wintermute" else (
              r["from_entity"] if r["to_entity"]=="Wintermute" else None),
    axis=1
)

# Merge with classification results
merged = df.merge(
    results_df.rename(columns={"input": "counterparty"}),
    on="counterparty",
    how="left"
)

# Filter: only DeFi interactions
df_defi = merged[merged["classification"]=="DeFi"]

df_defi


,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number,counterparty,match,score,classification
0,2025-04-08 12:49:57+00:00,0xB1026b8e7276e7AC75410F1fcbbe21796e8f7526,Camelot,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0,Camelot,Camelot V3,18.5,DeFi
1,2025-04-08 12:49:54+00:00,0xcDa53B1F66614552F834cEeF361A8D12a0B8DaD8,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0,Uniswap,Uniswap V3,18.5,DeFi
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59,Aerodrome Finance,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0,Aerodrome Finance,Thetanuts Finance,30.0,DeFi
3,2025-04-08 12:49:51+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,0x72AB388E2E2F6FaceF59E3C3FA2C4E29011c2D38,PancakeSwap,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0,PancakeSwap,PancakeSwap AMM,18.0,DeFi
4,2025-04-08 12:49:50+00:00,0x641C00A822e8b671738d32a431a4Fb6074E5c79d,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0,Uniswap,Uniswap V3,18.5,DeFi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436994,2025-04-08 01:30:11+00:00,0x478946BcD4a5a22b316470F5486fAfb928C0bA25,Velodrome Finance,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,USDC,2763.916721,2763.916721,0x372cd43e7eb18daa53e73d2a059c0dee68eeb53b0b67...,optimism,134238517.0,Velodrome Finance,Thetanuts Finance,30.0,DeFi
436995,2025-04-08 01:30:11+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,0xC6F780497A95e246EB9449f5e4770916DCd6396A,Uniswap,WETH,0.858818,1332.207678,0x0a5a54bca3a858a0597e72e5378ecc2a1d3ac5c3bd4d...,arbitrum_one,324056309.0,Uniswap,Uniswap V3,18.5,DeFi
436996,2025-04-08 01:30:11+00:00,0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,USDC,41275.972918,41275.972918,0x508676cc2178573aed6f5c20e36ec470b66c4d03d1f5...,ethereum,22220904.0,Uniswap,Uniswap V3,18.5,DeFi
436998,2025-04-08 01:30:11+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,0x7fCDC35463E3770c2fB992716Cd070B63540b947,PancakeSwap,WETH,0.132825,206.039373,0x2df77ad6d13e9d4cbf89b46a4e19c69eb02106a7b00b...,arbitrum_one,324056309.0,PancakeSwap,PancakeSwap AMM,18.0,DeFi


### Identifying Wintermute’s Most Actively Traded Tokens

With the dataset filtered to DeFi-related interactions, the next step is to quantify which tokens Wintermute trades most frequently and which represent the largest economic flows.

Two complementary measures are computed:

1. **Transaction frequency** – the number of times each token appears in DeFi-related transfers involving Wintermute.
2. **Economic volume (USD)** – the total USD value transferred for each token across all DeFi interactions.

These metrics reveal both high-velocity tokens and high-value tokens, offering insight into Wintermute’s trading priorities and liquidity engagement across DeFi markets.


In [6]:
# Most traded tokens by number of transactions
token_frequency = (
    df_defi
    .groupby("token")
    .size()
    .sort_values(ascending=False)
    .rename("tx_count")
)

# Most traded tokens by economic volume (in USD)
token_volume = (
    df_defi
    .groupby("token")["usd"]
    .sum()
    .sort_values(ascending=False)
    .rename("total_usd_volume")
)

print("Top tokens by frequency:\n", token_frequency.head(10))
print("\nTop tokens by USD volume:\n", token_volume.head(10))


Top tokens by frequency:
 token
USDC     110467
WSOL      97398
SOL       48350
WETH      41093
USDT      17276
WBTC      12783
cbBTC      8961
ARB        5335
WBNB       2798
JUP        2454
Name: tx_count, dtype: int64

Top tokens by USD volume:
 token
USDC     3.728163e+08
WETH     2.382804e+08
WSOL     8.355886e+07
cbBTC    7.592907e+07
USDT     6.290960e+07
WBTC     5.989094e+07
BTCB     1.287819e+07
WBNB     1.022826e+07
ETH      8.608820e+06
EURC     3.250701e+06
Name: total_usd_volume, dtype: float64


### Visualising Token Activity on a Log–Log Scale

To gain a broader view of Wintermute’s trading behaviour across all tokens, the dataset is aggregated to compute two key metrics for each asset:

- **Transaction frequency**, representing how often the token appears in Wintermute’s DeFi interactions.
- **Total USD volume**, capturing the cumulative economic value transferred.

These measures are combined into a single table and transformed using a base-10 logarithm, allowing both common low-volume tokens and a small number of extremely active or high-value tokens to be visualised on the same scale.

An interactive scatter plot is then generated, with each point representing a token. The x-axis shows the logarithm of transaction frequency and the y-axis shows the logarithm of the total USD volume. Hovering over a point reveals the token name along with its raw counts and economic volume, enabling closer inspection of outliers and dominant trading pairs. This representation provides an efficient way to identify clusters, high-impact tokens, and the overall structure of Wintermute’s DeFi trading footprint.


In [7]:
token_frequency = (
    df_defi
    .groupby("token")
    .size()
    .rename("tx_count")
)

token_volume = (
    df_defi
    .groupby("token")["usd"]
    .sum()
    .rename("total_usd_volume")
)

# Merge into a single frame
token_stats = (
    pd.concat([token_frequency, token_volume], axis=1)
    .reset_index()
    .rename(columns={"index": "token"})
)

# Apply log scale (add small constant to avoid log(0))
token_stats["log_tx"] = np.log10(token_stats["tx_count"] + 1)
token_stats["log_usd"] = np.log10(token_stats["total_usd_volume"] + 1)

# Interactive scatter plot
fig = px.scatter(
    token_stats,
    x="log_tx",
    y="log_usd",
    hover_name="token",
    hover_data={
        "tx_count": True,
        "total_usd_volume": ":,.2f",
    },
    labels={
        "log_tx": "log₁₀(Transaction Frequency)",
        "log_usd": "log₁₀(Total USD Volume)",
    },
    title="Wintermute DeFi Token Activity — Log–Log Scatter Plot",
)

fig.update_traces(marker=dict(size=10, opacity=0.8))

fig.show()


In [ ]:
X = token_stats[["log_tx", "log_usd"]].to_numpy()

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
token_stats["cluster"] = kmeans.fit_predict(X)
token_stats["cluster"] = token_stats["cluster"].astype(str)


fig = px.scatter(
    token_stats,
    x="tx_count",
    y="total_usd_volume",
    color="cluster",      
    hover_name="token",
    hover_data={
        "tx_count": True,
        "total_usd_volume": True,
        "log_tx": False,
        "log_usd": False,
        "cluster": True
    },
    title="Wintermute – Token Activity Clusters (Frequency vs USD Volume)"
)

fig.update_layout(
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="Transaction Frequency (log scale)",
    yaxis_title="USD Volume (log scale)",
)

fig.show()


cluster_summary = (
    token_stats
    .groupby("cluster")[["tx_count", "total_usd_volume"]]
    .agg(["mean", "sum", "count"])
)

cluster_summary["mean_usd_per_tx"] = (
    cluster_summary["total_usd_volume"]["sum"] / cluster_summary["tx_count"]["sum"]
)

cluster_summary


cluster_summary

tx_count               total_usd_volume                      \
                 mean     sum count             mean           sum count   
cluster                                                                    
0          127.731707   10474    82     2.439891e+05  2.000711e+07    82   
1            6.769231     176    26     2.001371e+03  5.203565e+04    26   
2        13351.296296  360485    27     3.505332e+07  9.464395e+08    27   

        mean_usd_per_tx  
                         
cluster                  
0           1910.169013  
1            295.657080  
2           2625.461619

### Identifying the Most Active Blockchains

In order to understand where Wintermute conducts the majority of its DeFi activity, the dataset is grouped by blockchain network. Two complementary metrics are calculated:

- **Transaction frequency**, indicating how many Wintermute-related DeFi transfers occur on each chain.
- **Total USD volume**, measuring the cumulative economic value moved on each network.

These metrics highlight which chains serve as Wintermute’s primary venues for trading and liquidity provision, distinguishing high-throughput environments from those associated with larger-value transfers. This provides a clearer picture of Wintermute’s cross-chain operational footprint within the DeFi ecosystem.


In [14]:
chain_frequency = (
    df_defi
    .groupby("chain")
    .size()
    .sort_values(ascending=False)
    .rename("tx_count")
)

chain_volume = (
    df_defi
    .groupby("chain")["usd"]
    .sum()
    .sort_values(ascending=False)
    .rename("total_usd_volume")
)

print("Most active chains (frequency):\n", chain_frequency.head(10))
print("\nMost active chains (volume):\n", chain_volume.head(10))


Most active chains (frequency):
 chain
solana          253651
arbitrum_one     67100
base             20750
ethereum         18874
bsc               7397
optimism          3363
polygon              1
Name: tx_count, dtype: int64

Most active chains (volume):
 chain
ethereum        5.060332e+08
solana          1.966697e+08
base            1.579900e+08
arbitrum_one    5.947468e+07
bsc             3.897393e+07
optimism        7.357098e+06
polygon         0.000000e+00
Name: total_usd_volume, dtype: float64


### Visualising Wintermute’s Cross-Chain Activity

To illustrate the distribution of Wintermute’s DeFi interactions across different blockchain networks, two pie charts are constructed using the aggregated chain statistics:

- **Transaction frequency distribution** shows how Wintermute’s activity is spread across chains in terms of the number of executed transfers.
- **USD volume distribution** reveals which networks handle the largest share of Wintermute’s economic flow.

 This visual comparison highlights the networks where Wintermute is most active and those that dominate in terms of value transferred, offering insight into the firm’s strategic positioning within the broader multichain DeFi landscape.


In [ ]:
# Convert to DataFrame for Plotly
df_chain_freq = chain_frequency.reset_index()
df_chain_vol = chain_volume.reset_index()

palette = px.colors.qualitative.Dark24

# Create a consistent mapping from chain name → colour
unique_chains = sorted(set(df_chain_freq["chain"]).union(df_chain_vol["chain"]))
color_map = {chain: palette[i % len(palette)] for i, chain in enumerate(unique_chains)}

# Pie Chart – Transaction Frequency
fig1 = px.pie(
    df_chain_freq,
    names="chain",
    values="tx_count",
    title="Wintermute – Chain Activity by Transaction Frequency",
    color="chain",
    color_discrete_map=color_map,
    hole=0.4
)
fig1.update_traces(textposition="inside", textinfo="percent+label")
fig1.show()

# Pie Chart – USD Volume
fig2 = px.pie(
    df_chain_vol,
    names="chain",
    values="total_usd_volume",
    title="Wintermute – Chain Activity by USD Volume",
    color="chain",
    color_discrete_map=color_map,
    hole=0.4
)
fig2.update_traces(textposition="inside", textinfo="percent+label")
fig2.show()

### Identifying Wintermute’s Most Active DeFi Platforms

To understand which decentralised platforms Wintermute interacts with most frequently, the dataset is grouped by the identified protocol counterparty. Two complementary metrics are computed:

- **Interaction frequency** — the number of Wintermute transactions involving each DeFi platform.
- **Total USD volume** — the cumulative economic value of those interactions.

These measures highlight both high-volume and high-value relationships, providing insight into where Wintermute concentrates its DeFi activity and which platforms serve as its main liquidity venues.


In [16]:
# Frequency of interactions per protocol
platform_freq = df_defi["counterparty"].value_counts()

# USD volume per protocol
platform_volume = (
    df_defi.groupby("counterparty")["usd"]
           .sum()
           .sort_values(ascending=False)
)

print("Most interacted DeFi platforms (count):\n")
print(platform_freq.head(10))

print("\nHighest USD volume platforms:\n")
print(platform_volume.head(10))

Most interacted DeFi platforms (count):

Orca                         84105
Raydium                      82780
Uniswap                      51477
Jito                         48343
Meteora (Prev. Mercurial)    31702
PancakeSwap                  21608
Fluid (Instadapp)            13933
Camelot                      10729
Aerodrome Finance             9002
Phoenix                       6266
Name: counterparty, dtype: int64

Highest USD volume platforms:

counterparty
Uniswap                      3.573379e+08
Aerodrome Finance            1.116319e+08
Coinbase                     1.099160e+08
Orca                         8.665894e+07
CoW Protocol                 6.933769e+07
Meteora (Prev. Mercurial)    6.704490e+07
Fluid (Instadapp)            5.108056e+07
PancakeSwap                  4.628817e+07
Raydium                      2.121679e+07
Gate.io                      1.203993e+07
Name: usd, dtype: float64


### Visualising DeFi Platform Activity on a Log–Log Scale

To assess Wintermute’s interaction patterns across DeFi platforms, the transaction frequency and total USD volume associated with each protocol are combined into a single dataset. This provides a comprehensive view of both how often Wintermute interacts with a platform and the economic significance of those interactions.

A logarithmic transformation is applied to both metrics to accommodate the wide range of activity levels, allowing smaller platforms and high-volume outliers to be displayed together meaningfully. An interactive scatter plot is then generated, with each point representing a DeFi platform. Hovering over a point reveals the raw transaction count and total USD volume, enabling quick inspection of both dominant platforms and secondary liquidity venues.

This visualisation offers an intuitive overview of Wintermute’s DeFi footprint, highlighting clusters of frequently used platforms and identifying those that command particularly h


In [ ]:
# Frequency of interactions per protocol
platform_freq = df_defi["counterparty"].value_counts()

# USD volume per protocol
platform_volume = (
    df_defi.groupby("counterparty")["usd"]
           .sum()
           .sort_values(ascending=False)
)

# Combine into a single DataFrame
platform_stats = (
    pd.concat([platform_freq, platform_volume], axis=1)
      .reset_index()
      .rename(columns={
          "index": "platform",
          "counterparty": "tx_count",
          "usd": "total_usd_volume"
      })
)

# Log scale features (add +1 to avoid log(0))
platform_stats["log_tx"] = np.log10(platform_stats["tx_count"] + 1)
platform_stats["log_usd"] = np.log10(platform_stats["total_usd_volume"] + 1)

# Interactive scatter plot 
fig = px.scatter(
    platform_stats,
    x="log_tx",
    y="log_usd",
    hover_name="platform",
    hover_data={
        "tx_count": True,
        "total_usd_volume": ":,.2f",
    },
    labels={
        "log_tx": "log₁₀(Transaction Frequency)",
        "log_usd": "log₁₀(Total USD Volume)",
    },
    title="Wintermute DeFi Platform Activity — Log–Log Scatter Plot",
)

# Style: single colour + no legend
fig.update_traces(
    marker=dict(size=10, opacity=0.8, color="#1f77b4"),
    showlegend=False
)

fig.show()


### Clustering DeFi Platforms by Activity Patterns

To identify structural patterns in Wintermute’s interactions across DeFi platforms, a clustering analysis is performed using K-means. Each platform is represented by two log-scaled features:

- **log₁₀(Transaction Frequency)** — how often Wintermute interacts with the platform.
- **log₁₀(Total USD Volume)** — the cumulative value transferred.

These features capture both the intensity and economic significance of Wintermute’s activity, enabling the clustering algorithm to group platforms with similar behavioural profiles.

Three clusters are generated, providing a high-level segmentation of Wintermute’s DeFi counterparties. The resulting clusters are visualised in a log–log scatter plot, with each point representing a platform and colour indicating its assigned cluster. This plot helps distinguish high-frequency platforms, high-value platforms, and those that receive relatively limited interaction.

A summary table is also computed, reporting the mean, total, and count of platforms within each cluster. This provides an aggregated view of how Wintermute’s DeFi relationships distribute across different activity tiers.


In [19]:
# K-means clustering in log–log space

X = platform_stats[["log_tx", "log_usd"]].to_numpy()

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
platform_stats["cluster"] = kmeans.fit_predict(X).astype(str)

# Interactive scatter plot

fig = px.scatter(
    platform_stats,
    x="tx_count",
    y="total_usd_volume",
    color="cluster",
    hover_name="platform",
    hover_data={
        "tx_count": True,
        "total_usd_volume": ":,.2f",
        "log_tx": False,
        "log_usd": False,
        "cluster": True,
    },
    title="Wintermute – DeFi Platform Activity Clusters (Frequency vs USD Volume)"
)

fig.update_layout(
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="Transaction Frequency (log scale)",
    yaxis_title="USD Volume (log scale)"
)

fig.show()

# Summary statistics for each cluster

cluster_summary = (
    platform_stats
    .groupby("cluster")[["tx_count", "total_usd_volume"]]
    .agg(["mean", "sum", "count"])
)

cluster_summary["mean_usd_per_tx"] = (
    cluster_summary["total_usd_volume"]["sum"] /
    cluster_summary["tx_count"]["sum"]
)

cluster_summary


tx_count               total_usd_volume                      \
                 mean     sum count             mean           sum count   
cluster                                                                    
0           83.545455     919    11     1.473612e+06  1.620974e+07    11   
1            4.750000      38     8     9.163388e+02  7.330711e+03     8   
2        20565.500000  370179    18     5.279342e+07  9.502816e+08    18   

        mean_usd_per_tx  
                         
cluster                  
0          17638.451165  
1            192.913439  
2           2567.086761

### Exploring Temporal Trends in Wintermute’s DeFi Activity

To examine how Wintermute’s interaction patterns evolve over time, an interactive visualisation tool is constructed. The timestamped DeFi dataset is resampled at a chosen temporal resolution and aggregated across one of three categories:

- **Token** – tracks value flows per asset over time.
- **Blockchain** – highlights shifts in activity across chains.
- **Platform** – shows how interactions with specific DeFi protocols vary.

A dropdown menu allows selection of both the time granularity (`Minute` or `Hour`) and the grouping category. For clarity, only the top ten entities by total USD volume within the selected category are displayed. The resulting line plot provides a dynamic view of where Wintermute’s value flows are concentrated at different points in time, revealing bursts of activity, quiet periods, and the relative prominence of different tokens, chains, or platforms throughout the observed window.


In [ ]:
# Prepare timestamp
df_defi = df_defi.copy()
df_defi["timestamp"] = pd.to_datetime(df_defi["timestamp"], utc=True)

# Interactive function

def plot_temporal_trend(time_granularity, category):
    
    # Choose the resample rule
    if time_granularity == "Minute":
        rule = "T"   
    elif time_granularity == "Hour":
        rule = "H"   

    # Choose the grouping field
    if category == "Token":
        field = "token"
    elif category == "Blockchain":
        field = "chain"
    elif category == "Platform":
        field = "counterparty"

    # Aggregate by time + category
    grouped = (
        df_defi
        .set_index("timestamp")
        .groupby(field)
        .resample(rule)["usd"]
        .sum()
        .reset_index()
    )

    # Show only top 10 by total value
    top_categories = (
        grouped.groupby(field)["usd"]
        .sum()
        .sort_values(ascending=False)
        .head(10)
        .index
    )
    
    filtered = grouped[grouped[field].isin(top_categories)]

    # Plot
    fig = px.line(
        filtered,
        x="timestamp",
        y="usd",
        color=field,
        title=f"Wintermute DeFi Activity Over Time — {category} by {time_granularity.lower()}",
        labels={"usd": "USD Volume", "timestamp": "Time"},
    )

    fig.update_layout(legend_title=category)
    fig.show()


# Dropdown widgets

time_dropdown = Dropdown(
    options=["Minute", "Hour"],
    value="Hour",
    description="Timeframe:"
)

category_dropdown = Dropdown(
    options=["Token", "Blockchain", "Platform"],
    value="Token",
    description="Group by:"
)

# Create interactive widget
interactive_plot = interact(
    plot_temporal_trend,
    time_granularity=time_dropdown,
    category=category_dropdown
)


interactive(children=(Dropdown(description='Timeframe:', index=1, options=('Minute', 'Hour'), value='Hour'), D…